In [1]:
import tensorflow
from pysparkling import H2OContext

In [3]:
## Demo-Specific configuration
NODES=3 # Number of partitions to split data into (~simulate a number of nodes)
DATASET_DIR="/users/nidhimehta/h2o-3/bigdata/laptop/mnist"

In [4]:
## Initialize TensorFlow session and test it
def map_fun(i):
  import tensorflow as tf
  with tf.Graph().as_default() as g:
    hello = tf.constant('Sparkling, TensorFlow!', name="hello_constant")
    with tf.Session() as sess:
      return sess.run(hello)
sc.parallelize(range(NODES), NODES).map(map_fun).collect()

['Sparkling, TensorFlow!', 'Sparkling, TensorFlow!', 'Sparkling, TensorFlow!']

In [5]:
## Read MNIST data into H2O
import h2o
h2o.__version__
hc = H2OContext(sc).start()
train_frame = h2o.import_file("{}/{}".format(DATASET_DIR, "train.csv.gz"))
test_frame = h2o.import_file("{}/{}".format(DATASET_DIR, "test.csv.gz"))

H2O cluster uptime:,8 seconds 19 milliseconds
H2O cluster version:,3.8.2.2
H2O cluster name:,sparkling-water-nidhimehta_-1766487378
H2O cluster total nodes:,3
H2O cluster total free memory:,2.88 GB
H2O cluster total cores:,24
H2O cluster allowed cores:,24
H2O cluster healthy:,True
H2O Connection ip:,192.168.1.100
H2O Connection port:,54337
H2O Connection proxy:,None



Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%


In [6]:
## Turn H2O DataFrame into a Spark DataFrame
train_df = hc.as_spark_frame(train_frame).repartition(NODES)
test_df = hc.as_spark_frame(test_frame).repartition(NODES)
#train_df.printSchema()

In [7]:
# Configure a TensorFlow Deep Learning model

# - it loads local training data into numpy array (from Spark -> Python)
# - train TF Deep Learning model with 1 hidden layer (50 neurons)
# - output accuracy on training data
def create_nn(data_train, data_test, iterations, batch_size):
    import tensorflow as tf
    # Symbolic computation
    x = tf.placeholder(tf.float32, [None, 784])
    W1 = tf.Variable(tf.random_normal([784, 50],stddev=0.1))
    W2 = tf.Variable(tf.random_normal([50, 10],stddev=0.1))
    b1 = tf.Variable(tf.random_normal([50],stddev=0.1))
    b2 = tf.Variable(tf.random_normal([10],stddev=0.1))
    hidden = tf.nn.relu(tf.matmul(x, W1) + b1)
    y = tf.nn.softmax(tf.matmul(hidden, W2) + b2)
    y_ = tf.placeholder(tf.float32, [None, 10])
    cross_entropy = -tf.reduce_sum(y_*tf.log(y))                    
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    
    # Initialize TF
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    print("Training TensorFlow Deep Learning model")
    for i in range(iterations):
      #print("TensorFlow iter: ", i, " session: ", sess)
      batch_xs, batch_ys = data_train.next_batch(batch_size)
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        
    model = [(sess.run(W1),sess.run(W2),sess.run(b1),sess.run(b2))]

    # Model evaluation
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    batch_xs, batch_ys = data_test.next_batch(1000)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Training Accuracy:", sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
    #print(sess.run(tf.argmax(y,1), feed_dict={x: batch_xs, y_: batch_ys}))
    
    sess.close()
    return iter(model)
    
    # Export the model
    #from tensorflow_serving.session_bundle import exporter
    #export_path = "/tmp/xxx/"
    #saver = tf.train.Saver(sharded=True)
    #model_exporter = exporter.Exporter(saver)
    #signature = exporter.classification_signature(input_tensor=x, scores_tensor=y)
    #model_exporter.init(sess.graph.as_graph_def(), default_graph_signature=signature)
    #model_exporter.export(export_path, tf.constant(FLAGS.export_version), sess)
    
## Internal Helpers

# Sampling with replacement to provide a batch size
# Load everything into numpy datastructure
import numpy as np

def expand1hot(response, levels):
    nrows = response.shape[0]
    result = np.zeros((nrows, levels), dtype=np.float32)
    result[np.arange(nrows), response.astype(np.int8)] = 1.0
    return result

class RowData:
    def __init__(self, it):
        self._part_array = np.array([ [a for a in x] for x in it], dtype=np.float32)
        # Definition of input features
        self._x = range(784)
        # Index of response
        self._y = 784

    def next_batch(self, n):
        # Sample from local data without replacement
        dim = self._part_array.shape[0] # number of rows
        sample = np.random.choice(dim, n, replace=False)
        data = self._part_array[sample, :]
        # Data comming from H2O, pixel values are 0..255
        # FIXME: this should be done via RDD or H2O API directly !
        train = data[:, self._x]/255 
        response = expand1hot(data[:, self._y], 10)
        return (train, response)



In [8]:
## Run TensorFlow on each node

# Number of iterations
ITERATIONS = 100
# Batch size
BATCH_SIZE = 100
# Use Mnist dataset provided by TensorFlow - for debugging only
USE_TF_MNIST=False

def train_nn(iterations, batch_size, use_tf_mnist=False):
    def perPartition(it):
        if not use_tf_mnist:
            train_data = RowData(it)
            test_data = train_data
        else:
            from tensorflow.examples.tutorials.mnist import input_data
            mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
            train_data = mnist.train
            test_data = mnist.train
            
        return create_nn(train_data, test_data, iterations, batch_size)
        
    return perPartition

modelsPerNode = train_df.mapPartitions(train_nn(ITERATIONS, BATCH_SIZE, USE_TF_MNIST)).collect()

In [9]:
# Average the weights and biases across all node-local models
W1 = modelsPerNode[0][0]
W2 = modelsPerNode[0][1]
b1 = modelsPerNode[0][2]
b2 = modelsPerNode[0][3]

AVERAGE = True

if (AVERAGE):
  for i in range(1,NODES):
    W1 = W1 + modelsPerNode[i][0]
    W2 = W2 + modelsPerNode[i][1]
    b1 = b1 + modelsPerNode[i][2]
    b2 = b2 + modelsPerNode[i][3]

W1 = W1/NODES
W2 = W2/NODES
b1 = b1/NODES
b2 = b2/NODES

#print(W1)
#print(W2)
print(b1)
print(b2)

[-0.06560349 -0.09738561 -0.06779324  0.17688142 -0.00221774  0.07548282
 -0.00934662 -0.00779513  0.14491893  0.20652492 -0.05385517 -0.0494847
  0.06340139 -0.06675905  0.02318799 -0.00206788 -0.10062058  0.01637787
 -0.04579215  0.02075566  0.00107423 -0.0511154  -0.05652    -0.02502443
 -0.05336539  0.12621792  0.01538029  0.0035066  -0.0599067  -0.03444355
  0.16539933 -0.16662456 -0.02543416 -0.01268842 -0.1143443  -0.03024692
 -0.0537156   0.20251174  0.02087393  0.15216446 -0.09731451 -0.05295835
 -0.09113394  0.05675854  0.06116894 -0.01474935 -0.04777083 -0.0273279
  0.05949728 -0.09491964]
[-0.34054735 -0.25906125 -0.12921773 -0.08484312  0.20129819  0.32576436
 -0.28236356 -0.11957812  0.39435092  0.26485518]


In [10]:
W1 = np.transpose(W1)
W2 = np.transpose(W2)
b1 = np.matrix(b1)
b1 = np.transpose(b1)
b2 = np.matrix(b2)
b2 = np.transpose(b2)

In [11]:
#Initialize an H2O Model with those weights/biases
#from pysparkling import *
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

## Create an H2O Deep Learning model from the TensorFlow model
dlmodel = H2ODeepLearningEstimator(
    hidden=[50],             ## same Network layout as TF - one hidden layer
    epochs=0,                ## no training done in H2O - just copy over the model from TF
    ignore_const_cols=False  ## keep all input features (unless we also drop const cols in TF)
    
    ### Initialize the H2O model with the TensorFlow model state
    ### Requires H2O 3.8.2.1 or later
    ,initial_weights=[h2o.H2OFrame(W1.tolist()),h2o.H2OFrame(W2.tolist())]
    ,initial_biases=[h2o.H2OFrame(b1.tolist()),h2o.H2OFrame(b2.tolist())]
)
train_frame[784] = train_frame[784].asfactor()
dlmodel.train(x=list(range(784)),y=784,training_frame=train_frame)


Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

deeplearning Model Build Progress: [                                                  ] 00%


In [12]:
## Check the model performance - Will only be good if we actually copied the TF model into the H2O model above
dlmodel.model_performance(train_frame)


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.238783078818
R^2: 0.971395497551
LogLoss: 0.937801596691

Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
5369.0,2.0,26.0,45.0,29.0,361.0,5.0,9.0,27.0,50.0,0.0935337,"554 / 5,923"
1.0,5932.0,30.0,8.0,13.0,660.0,0.0,93.0,3.0,2.0,0.1201424,"810 / 6,742"
143.0,221.0,4725.0,117.0,259.0,174.0,5.0,201.0,101.0,12.0,0.2069486,"1,233 / 5,958"
26.0,62.0,199.0,4699.0,18.0,789.0,0.0,258.0,27.0,53.0,0.2335671,"1,432 / 6,131"
15.0,28.0,23.0,4.0,4642.0,943.0,1.0,95.0,2.0,89.0,0.2054091,"1,200 / 5,842"
78.0,24.0,23.0,94.0,62.0,5038.0,28.0,23.0,28.0,23.0,0.0706512,"383 / 5,421"
115.0,91.0,788.0,4.0,1408.0,612.0,2883.0,2.0,14.0,1.0,0.5128422,"3,035 / 5,918"
9.0,50.0,51.0,13.0,38.0,178.0,2.0,5643.0,1.0,280.0,0.0992817,"622 / 6,265"
138.0,1070.0,264.0,161.0,111.0,2074.0,21.0,164.0,1762.0,86.0,0.6988549,"4,089 / 5,851"
65.0,12.0,14.0,54.0,446.0,979.0,2.0,1450.0,14.0,2913.0,0.5103379,"3,036 / 5,949"



Top-10 Hit Ratios: 


k,hit_ratio
1,0.7267666
2,0.8734666
3,0.9399666
4,0.9653166
5,0.9803166
6,0.9895833
7,0.9949999
8,0.9980333
9,0.9991833
10,0.9999999


In [13]:
## Now we have a POJO for the TensorFlow model!
dlmodel.download_pojo()

Filepath: /DeepLearning_model_python_1461379919661_1.java
/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2016-04-22T20:54:04.881-06:00
  3.8.2.2
  
  Standalone prediction code with sample test data for DeepLearningModel named DeepLearning_model_python_1461379919661_1

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://192.168.1.100:54337/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://192.168.1.100:54337/3/Models.java/DeepLearning_model_python_1461379919661_1 > DeepLearning_model_python_1461379919661_1.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m DeepLearning_model_python_1461379919661_1.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
import java.util.Map;
import hex.genmodel.GenModel;
import hex.genmodel.annotations.ModelPojo;

@ModelPojo(name="DeepLearning_model_python_1461